In [2]:
%load_ext autoreload
%matplotlib inline

In [43]:
import scipy.io as sio
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import random
import json
import math
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.utils import shuffle
from collections import defaultdict
# change data_dir according to the specific configuration
choice = [ 'Processed_Data_1Hz','Processed_Data_1Hz_Denoised', 'Processed_Data']
data_dir = '/media/scratch/yding/lgfs/eeg_classification/raw/Bike_EEG_Data/'+choice[1]+'/'
printdata = lambda dd : print(json.dumps(store, indent=2))

In [4]:
'''
stores data conveniently
{
cond 1: {
    imageType1: {
        sj1 : [ndarray],
        sj2 : [ndarray],
        ...
        }
    }
    imageType:2 : {
    
    }
    ...
...    
}

'''
store = defaultdict(lambda: {})
total_subjects = 12
subject_nums = ['04','06','07','08','09','10','11','12','13','14','15','16']
conditions = ['bikeHigh', 'bikeLow','resting']
for cond in conditions:
    store[cond]['target'] = {}
    store[cond]['distractor'] = {}
    store[cond]['standard'] = {}
    for sj_num in subject_nums:
        current_data = sio.loadmat(data_dir + "sj" + sj_num + cond + "Rsvp1Hz.mat")
        # transpose from channel, time, trial to trial, channel, time
        target = np.transpose(current_data['tarEEG'], axes=[2, 0, 1])
        distractor = np.transpose(current_data['disEEG'], axes=[2, 0, 1])
        standard = np.transpose(current_data['stdEEG'], axes=[2, 0, 1])
        store[cond]['target'][sj_num] = target
        store[cond]['distractor'][sj_num] = distractor
        store[cond]['standard'][sj_num] = standard
        
for cond in conditions:
    for imageType in store[cond]:
        for i in subject_nums:
            cur = store[cond][imageType]
            allExceptOne = np.vstack([cur[j] for j in subject_nums if j != i])
            X = np.vstack((allExceptOne, cur[i]))
            Y = np.array([0] * allExceptOne.shape[0] + [1] * cur[i].shape[0])
            X, Y = shuffle(X, Y, random_state = 420)            
            train80 = math.floor(.8 * len(Y))
            
            train = {
                'X': X[:train80],
                'y': train_Y[:train80]
            }
            test = {
                'X': X[train80:],
                'y': Y[train80:]
            }
            print(f"auth/train_{cond}_{imageType}_{i}")
            np.save(f"auth/train_{cond}_{imageType}_{i}", train)
            np.save(f"auth/test_{cond}_{imageType}_{i}", test)

    

Target: (100, 40, 128), Distractor: (100, 40, 128), Standard: (800, 40, 128)
Target: (100, 40, 128), Distractor: (100, 40, 128), Standard: (800, 40, 128)
Target: (100, 40, 128), Distractor: (100, 40, 128), Standard: (800, 40, 128)
Target: (100, 40, 128), Distractor: (100, 40, 128), Standard: (800, 40, 128)
Target: (100, 40, 128), Distractor: (100, 40, 128), Standard: (800, 40, 128)
Target: (100, 40, 128), Distractor: (100, 40, 128), Standard: (800, 40, 128)
Target: (99, 40, 128), Distractor: (100, 40, 128), Standard: (793, 40, 128)
Target: (100, 40, 128), Distractor: (100, 40, 128), Standard: (800, 40, 128)
Target: (100, 40, 128), Distractor: (100, 40, 128), Standard: (800, 40, 128)
Target: (100, 40, 128), Distractor: (100, 40, 128), Standard: (800, 40, 128)
Target: (100, 40, 128), Distractor: (100, 40, 128), Standard: (800, 40, 128)
Target: (100, 40, 128), Distractor: (100, 40, 128), Standard: (800, 40, 128)
Target: (100, 40, 128), Distractor: (100, 40, 128), Standard: (800, 40, 128)


In [50]:
'''
stores data coarsely:
{
cond 1: {
        sj1 : [ndarray],
        sj2 : [ndarray],
        ...
        }
    }
    
    ...
...    
}

'''
store = defaultdict(lambda: {})
total_subjects = 12
subject_nums = ['04','06','07','08','09','10','11','12','13','14','15','16']
conditions = ['bikeHigh', 'bikeLow','resting']
for cond in conditions:
    store[cond]['target'] = {}
    store[cond]['distractor'] = {}
    store[cond]['standard'] = {}
    for sj_num in subject_nums:
        current_data = sio.loadmat(data_dir + "sj" + sj_num + cond + "Rsvp1Hz.mat")
        # transpose from channel, time, trial to trial, channel, time
        target = np.transpose(current_data['tarEEG'], axes=[2, 0, 1])
        distractor = np.transpose(current_data['disEEG'], axes=[2, 0, 1])
        standard = np.transpose(current_data['stdEEG'], axes=[2, 0, 1])
        store[cond][sj_num] = shuffle(np.concatenate((target, distractor, standard), axis=0), random_state=420)
        
for cond in conditions:
        for i in subject_nums:
            cur = store[cond]
            allExceptOne = np.vstack([cur[j] for j in subject_nums if j != i])
            X = np.vstack((allExceptOne, cur[i]))
            Y = np.array([0] * allExceptOne.shape[0] + [1] * cur[i].shape[0])
            X, Y = shuffle(X, Y, random_state = 420)            
            train80 = math.floor(.8 * len(Y))
            
            train = {
                'X': X[:train80],
                'y': train_Y[:train80]
            }
            test = {
                'X': X[train80:],
                'y': Y[train80:]
            }
            print(f"coarseauth/train_{cond}_{i}")
            np.save(f"coarseauth/train_{cond}_{i}", train)
            np.save(f"coarseauth/test_{cond}_{i}", test)

    

coarseauth/train_bikeHigh_04
coarseauth/train_bikeHigh_06
coarseauth/train_bikeHigh_07
coarseauth/train_bikeHigh_08
coarseauth/train_bikeHigh_09
coarseauth/train_bikeHigh_10
coarseauth/train_bikeHigh_11
coarseauth/train_bikeHigh_12
coarseauth/train_bikeHigh_13
coarseauth/train_bikeHigh_14
coarseauth/train_bikeHigh_15
coarseauth/train_bikeHigh_16
coarseauth/train_bikeLow_04
coarseauth/train_bikeLow_06
coarseauth/train_bikeLow_07
coarseauth/train_bikeLow_08
coarseauth/train_bikeLow_09
coarseauth/train_bikeLow_10
coarseauth/train_bikeLow_11
coarseauth/train_bikeLow_12
coarseauth/train_bikeLow_13
coarseauth/train_bikeLow_14
coarseauth/train_bikeLow_15
coarseauth/train_bikeLow_16
coarseauth/train_resting_04
coarseauth/train_resting_06
coarseauth/train_resting_07
coarseauth/train_resting_08
coarseauth/train_resting_09
coarseauth/train_resting_10
coarseauth/train_resting_11
coarseauth/train_resting_12
coarseauth/train_resting_13
coarseauth/train_resting_14
coarseauth/train_resting_15
coarseau

array([0, 1, 0, 1, 0, 0, 0, 0, 1, 0])